In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2019.9.11          |           py36_0         147 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    ca-certificates-2019.9.11  |       hecc5488_0         144 KB  conda-forge
    openssl-1.1.1d             |       h516909a_0         2.1 MB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0         conda-forge
    geopy:           1.20.0-py_0       conda-forge

The following packages will be UPDATED:

    cer

In [7]:
url = 'http://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

df = pd.read_html(url)

df=df[0]

df.head()


,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [8]:
df=df[df.Borough != 'Not assigned']
df = df.sort_values(by=['Postcode'])

df.reset_index(inplace=True)
df.drop('index',axis=1,inplace=True)

df.head(100)




,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,Rouge
1,M1B,Scarborough,Malvern
2,M1C,Scarborough,Port Union
3,M1C,Scarborough,Rouge Hill
4,M1C,Scarborough,Highland Creek
5,M1E,Scarborough,Guildwood
6,M1E,Scarborough,Morningside
7,M1E,Scarborough,West Hill
8,M1G,Scarborough,Woburn
9,M1H,Scarborough,Cedarbrae


In [9]:
df=df.groupby(['Postcode','Borough'])['Neighbourhood'].apply(list)
df=df.sample(frac=1).reset_index()
df['Neighbourhood']=df['Neighbourhood'].str.join(',')
df.head(500)


,Postcode,Borough,Neighbourhood
0,M6K,West Toronto,"Exhibition Place,Brockton,Parkdale Village"
1,M1R,Scarborough,"Maryvale,Wexford"
2,M2H,North York,Hillcrest Village
3,M8W,Etobicoke,"Long Branch,Alderwood"
4,M3M,North York,Downsview Central
5,M6N,York,"The Junction North,Runnymede"
6,M5N,Central Toronto,Roselawn
7,M8Z,Etobicoke,"Kingsway Park South West,Mimico NW,The Queensw..."
8,M5T,Downtown Toronto,"Kensington Market,Chinatown,Grange Park"
9,M4X,Downtown Toronto,"St. James Town,Cabbagetown"


In [10]:
df.shape

(103, 3)

In [11]:
geo_df = pd.read_csv('http://cocl.us/Geospatial_data')


geo_df.head(100)

# google python merge two files

# https://codescracker.com/python/program/python-program-merge-two-files.htm

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
5,M1J,43.744734,-79.239476
6,M1K,43.727929,-79.262029
7,M1L,43.711112,-79.284577
8,M1M,43.716316,-79.239476
9,M1N,43.692657,-79.264848


In [12]:
merged_inner = pd.merge(left=df,right=geo_df, left_on='Postcode', right_on='Postal Code')
merged_inner.shape
dfmerged = merged_inner.drop("Postal Code", axis=1)
dfmerged = dfmerged.rename(columns={"Postcode": "PostalCode"})
dfmerged.head(100)

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M6K,West Toronto,"Exhibition Place,Brockton,Parkdale Village",43.636847,-79.428191
1,M1R,Scarborough,"Maryvale,Wexford",43.750072,-79.295849
2,M2H,North York,Hillcrest Village,43.803762,-79.363452
3,M8W,Etobicoke,"Long Branch,Alderwood",43.602414,-79.543484
4,M3M,North York,Downsview Central,43.728496,-79.495697
5,M6N,York,"The Junction North,Runnymede",43.673185,-79.487262
6,M5N,Central Toronto,Roselawn,43.711695,-79.416936
7,M8Z,Etobicoke,"Kingsway Park South West,Mimico NW,The Queensw...",43.628841,-79.520999
8,M5T,Downtown Toronto,"Kensington Market,Chinatown,Grange Park",43.653206,-79.400049
9,M4X,Downtown Toronto,"St. James Town,Cabbagetown",43.667967,-79.367675


In [22]:
Downtown_Toronto_data = dfmerged[dfmerged['Borough'] == 'Downtown Toronto'].reset_index(drop=True)
Downtown_Toronto_data.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M5T,Downtown Toronto,"Kensington Market,Chinatown,Grange Park",43.653206,-79.400049
1,M4X,Downtown Toronto,"St. James Town,Cabbagetown",43.667967,-79.367675
2,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
3,M5L,Downtown Toronto,"Commerce Court,Victoria Hotel",43.648198,-79.379817
4,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636


In [23]:
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [24]:
# create map of Manhattan using latitude and longitude values
map_Toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(Downtown_Toronto_data['Latitude'], Downtown_Toronto_data['Longitude'], Downtown_Toronto_data['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Toronto)  
    
map_Toronto